# Orientación fotos

El primer problema que tenemos que resolver si queremos hacer un vídeo con las fotografías de AstroPi, es el de la corrección de la orientación de las mismas. AstroPi está colocada de forma que la cámara mira hacia la Tierra de forma lo más perpendicular posible, pero no es fácil conseguir orientarla de manera que el norte de la Tierra quede en la parte superior de las fotos para que éstas encajen en los mapas terrestres, además de que por la forma de la órbita esta orientación cambia continuamente. En realidad no cambia ya que la estación espacial no gira sobre sus ejes a no ser que haya una corrección de la órbita u orientación, pero al utilizar una órbita no ecuatorial, la inclinación aparente de las fotos cambia continuamente como luego veremos.

Vamos a calcular la orientación de las fotografías tomadas por AstroPi para girarlas de forma que queden con la misma orientación de los mapas (norte arriba). Utilizaremos como ejemplo los datos de las fotos 493 y 494 del proyecto del año 2021-2022 que corresponden al sobrevuelo de Pamplona.

|Fóto #|Fecha|Archivo foto|Latitud|Longitud|Altitud|
|:-----|:----|:-----------|:------|:-------|:------|
|493|2022-04-26 10:14:03,314|atlantes_493.jpg|43,097612303689|-2,3949488058073|420980,867804757|
|494|2022-04-26 10:14:18,314|atlantes_494.jpg|42,5769655132425|-1,34409030873256|420935,392917282|

Necesitamos calcular dos ángulos:

* α: Ángulo respecto al ecuador de la línea que une los puntos de las coordenadas (latitud, longitud) asociadas a cada foto, que en principio debería coincidir con el punto central de la foto (luego veremos que en realidad no es así). Este ángulo cambiará continuamente entre foto y foto por la inclinación de la órbita.
* β: Ángulo de la línea que representa el desplazamiento aparente de un objeto referencia (por ejemplo una nube pequeña o un cabo de la costa) entre dos fotos consecutivas. Este ángulo se mantendrá constante siempre que no haya cambio en la orientación de la estación respecto a la Tierra.

El ángulo α podemos representarlo así:

![Ángulo alfa](images/alpha.png)

Para determinar el ángulo necesitamos conocer las dimensiones del triángulo anterior. En el gráfico ya hemos colocado las distancias que vamos a calcular ahora. El cálculo lo haremos a partir de la latitud y longitud de las fotografías, que como hemos comentado antes coincidirá aproximadamente con el centro de las imágenes.

El cálculo de las distancias lo hacemos con el siguiente bloque de código. La distancia entre 493 y 494 no coincide con la que resultaría de aplicar el teorema de Pitágoras a los catetos del triángulo rectángulo anterior, porque en realidad las tres distancias son curvas que siguen la superficie de la Tierra (la curva se llama [Ortodrómica](https://es.wikipedia.org/wiki/Ortodr%C3%B3mica) o [Great-circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) en inglés).

In [3]:
# https://www.cosmoscalibur.com/blog/calcular-distancia-geodesica-con-python/
# https://geopy.readthedocs.io/en/latest/#module-geopy.distance
from geopy import distance

pnt_493 = (43.097612303689, -2.3949488058073)
pnt_494 = (42.5769655132425, -1.34409030873256)
pnt_X = (43.097612303689, -1.34409030873256)

print(distance.great_circle(pnt_493, pnt_494).m)
print(distance.great_circle(pnt_493, pnt_X).m)
print(distance.great_circle(pnt_X, pnt_494).m)

103408.22860916496
85322.4488809735
57893.363454430306


Con las distancias horizontal y vertical podemos calcular α con la función arcotangente de la siguiente forma:

$$
\alpha = \arctan{\left(\frac{-57893.363454430306}{85322.4488809735}\right)}
$$

Utilizamos la función [math.atan2()](https://docs.python.org/3/library/math.html#math.atan2) de Python para obtener la arcotangente:

In [4]:
import math

delta_rad = math.atan2(-57893.363454430306, 85322.4488809735)
delta_deg = math.degrees(delta_rad)
print(delta_deg)

-34.15784952082556


Por tanto:

$$
\alpha = \arctan{\left(\frac{-57893.363454430306}{85322.4488809735}\right)} = -34,16º
$$

Para el ángulo β usaremos algún elemento reconocible en las dos fotos consecutivas, cuanto más cerca del centro de las fotos mejor para evitar deformaciones de perspectiva. Por ejemplo:

![493](images/atlantes_493_crosshair.jpg)
![494](images/atlantes_494_crosshair.jpg)

Superponiendo las dos fotografías podemos ver el desplazamiento del punto elegido y por tanto podremos estimar tanto la distancia recorrida como el ángulo.

![494-493](images/atlantes_493_494_line.jpg)

El desplazamiento en píxeles entre los dos puntos es: 633 en horizontal hacia la derecha y 77 en vertical hacia abajo. El ángulo β se puede calcular de nuevo con la función arcotangente aplicada al cociente de las dos distancias:

$$
\beta = \arctan{\left(\frac{-77}{633}\right)} = -6,94º
$$

Ya era muy evidente analizando las fotos, que éstas están giradas casi 180º, es decir que la parte de arriba mira casi directamente hacia el sur. Si la cámara de la AstroPi hubiera estado perfectamente alineada con el eje de la Tierra (dejando arriba el norte), ambos ángulos α y β tendrían una diferencia entre ellos de 180º, ya que si la ISS avanza por ejemplo hacia el este en horizontal, lo que vemos en las fotos debería desplazarse hacia el oeste. Así pues, podemos obtener el ángulo de giro de la AstroPi respecto al norte, y por tanto el ángulo que deberemos girar en sentido contrario las fotos para orientarlas correctamente, por medio de la siguiente fórmula:

$$
\gamma = \alpha - \beta + 180 = (-34,16) - (-6,94) + 180 = 152,78º
$$

Rotando la foto 493 152,78º en el sentido contrario de las agujas del reloj vemos que efectivamente hace coincidir el perfil de la costa con lo que vemos en los mapas:

![493 girada](images/atlantes_493_girada.jpg)

![493 mapa](images/493_mapa.png)

Vamos a hacer los mismos cálculos sobre un par de fotos en un punto de la órbita alejado del anterior, para comprobar si la orientación de la cámara varía continuamente o no. Elegimos las fotos 452 y 453 en las que se sobrevoló el golfo de San Lorenzo en Canadá.

|Fóto #|Fecha|Archivo foto|Latitud|Longitud|Altitud|
|:-----|:----|:-----------|:------|:-------|:------|
|452|2022-04-26 10:03:48,314|atlantes_452.jpg|50,68999851209267|-58,40875937680981|420259,16469120927|
|453|2022-04-26 10:04:03,314|atlantes_453.jpg|50,890528630928614|-56,974697609781686|420350,4832888126|

Los cálculos de distancias del triángulo para determinar el ángulo α son los siguientes:

In [5]:
# https://www.cosmoscalibur.com/blog/calcular-distancia-geodesica-con-python/
# https://geopy.readthedocs.io/en/latest/#module-geopy.distance
from geopy import distance

pnt_452 = (50.68999851209267, -58.40875937680981)
pnt_453 = (50.890528630928614, -56.974697609781686)
pnt_X = (50.68999851209267, -56.974697609781686)

print(distance.great_circle(pnt_452, pnt_453).m)
print(distance.great_circle(pnt_452, pnt_X).m)
print(distance.great_circle(pnt_X, pnt_453).m)

103239.6601730321
101019.26542899004
22297.96335318471


$$
\alpha = \arctan{\left(\frac{22297.96335318471}{101019.26542899004}\right)} = 12,45º
$$

![452-453](images/atlantes_452_453_line.jpg)

Para el ángulo β el desplazamiento en píxeles entre los dos puntos es: 634 en horizontal hacia la derecha y 46 en vertical hacia abajo. El ángulo β se puede calcular de nuevo con la función arcotangente aplicada al cociente de las dos distancias:

$$
\beta = \arctan{\left(\frac{-46}{634}\right)} = -4,15º
$$

El ángulo β anterior debería coincidir con el calculado entre las fotos 493 y 494. De hecho vemos que la diferencia es pequeña (-6,94º frente a -4,15º) y en general atribuible a las distorsiones de perspectiva que aumentan conforme nos alejamos del punto central de la foto.

Con α y β ya podemos calcular γ o ángulo que tendremos que girar la foto:

$$
\gamma = \alpha - \beta + 180 = 12,45 - (-4,15) + 180 = 196,6º
$$

La comparación entre la foto girada y un mapa de la zona confirma que el cálculo es correcto (en la foto hay una especie de banquisa de hielo o nube alargada frente a la costa norte que complica la interpretación de esta orilla):

![452 girada](images/atlantes_452_girada.jpg)

![452 mapa](images/452_mapa.png)

Vemos pues que la orientación de las fotos varía durante la órbita, lo que nos obliga a calcularla foto a foto, por lo que tendremos que encontrar la forma de automatizar el cálculo, sobre todo el del ángulo β que se determina gráficamente. Como hemos comentando ya, este ángulo debería ser constante, por lo que calcularemos su valor entre todas las parejas de fotos (una foto y la siguiente) y luego haremos la media. Vamos a utilizar la mediana en lugar de la media ya que el sistema para calcular el desplazamiento en píxeles entre dos fotografías produce numerosos valores atípicos. Con la mediana conseguimos deshacernos de esos valores atípicos. El cálculo de β lo hacemos sobre el intervalo de fotos 445 a 622 con el siguiente bloque de código:

In [18]:
import os
import cv2
import math
import statistics

PHOTO_PATH = "atlantes_2021-2022"
PHOTO_START = 445
PHOTO_END = 622

def calculate_features(image_1_cv, image_2_cv, feature_number):
    orb = cv2.ORB_create(nfeatures = feature_number)
    keypoints_1, descriptors_1 = orb.detectAndCompute(image_1_cv, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(image_2_cv, None)
    return keypoints_1, keypoints_2, descriptors_1, descriptors_2

def calculate_matches(descriptors_1, descriptors_2):
    brute_force = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = brute_force.match(descriptors_1, descriptors_2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def display_matches(image_1_cv, keypoints_1, image_2_cv, keypoints_2, matches):
    match_img = cv2.drawMatches(image_1_cv, keypoints_1, image_2_cv, keypoints_2, matches[:10], None)
    resize = cv2.resize(match_img, (1600,600), interpolation = cv2.INTER_AREA)
    cv2.imshow('matches', resize)
    cv2.waitKey(0)
    cv2.destroyWindow('matches')

def find_matching_coordinates(keypoints_1, keypoints_2, matches):
    coordinates_1 = []
    coordinates_2 = []
    for match in matches:
        image_1_idx = match.queryIdx
        image_2_idx = match.trainIdx
        (x1,y1) = keypoints_1[image_1_idx].pt
        (x2,y2) = keypoints_2[image_2_idx].pt
        coordinates_1.append((x1,y1))
        coordinates_2.append((x2,y2))
    return coordinates_1, coordinates_2

path = os.path.dirname(os.path.realpath("__file__"))
photo_path = os.path.join(path, PHOTO_PATH)
beta_list = list()
for i in range(PHOTO_START, PHOTO_END):
    photo_a = os.path.join(photo_path, "atlantes_%d.jpg" % (i))
    photo_b = os.path.join(photo_path, "atlantes_%d.jpg" % (i+1))
    photo_a_cv = cv2.imread(photo_a, 0)
    photo_b_cv = cv2.imread(photo_b, 0)
    keypoints_a, keypoints_b, descriptors_a, descriptors_b = calculate_features(photo_a_cv, photo_b_cv, 1000)
    matches = calculate_matches(descriptors_a, descriptors_b)
    coordinates_a, coordinates_b = find_matching_coordinates(keypoints_a, keypoints_b, matches)
    beta_matches_list = list()
    for ca, cb in zip(coordinates_a, coordinates_b):
        dx = round(cb[0] - ca[0])
        dy = round(cb[1] - ca[1])
        beta_rad = math.atan2(-dy, dx)
        beta_deg = math.degrees(beta_rad)
        beta_matches_list.append(beta_deg)
        #print(f"H: {dx}; V: {dy}; β: {beta_deg}")
    #print(f"beta{i}_mediana: {statistics.median(beta_matches_list)}")
    beta_list.append(statistics.median(beta_matches_list))
beta = statistics.median(beta_list)
print(f"β = {beta}")

β = -5.889877764927383


Obtenemos pues el siguiente valor:

$$
\beta = -5,89º
$$

Como sabemos, debemos rotar las fotos un ángulo γ que se calcula así:

$$
\gamma = \alpha - \beta + 180 = \alpha + 185,89
$$

α variará entre foto y foto, por lo que lo determinaremos dinámicamente por código y aprovecharemos el mismo bucle para rotas las fotos. También vamos a almacenar en una estructura de datos los valores de α calculados para cada foto, puesto que nos hará falta a partir de ahora para casi cada operación que vayamos a hacer sobre las fotos. En realidad lo que más necesitaremos es el ángulo γ, pero puesto que se puede deducir directamente a partir de α nos quedamos solo con este último.

In [6]:
import os
import cv2
import numpy as np
import math
import exif
import json
from geopy import distance


def dms2dec(dms, ref):
    return (dms[0] + dms[1] / 60. + dms[2] / 3600.) * (1 if ref in ('N', 'E') else -1)

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

PHOTO_ORG_PATH = "atlantes_2021-2022"
PHOTO_ROT_PATH = "atlantes_2021-2022_rotadas"
PHOTO_START = 445
PHOTO_END = 622
path = os.path.dirname(os.path.realpath("__file__"))
beta = -5.889877764927383

photo_org_path = os.path.join(path, PHOTO_ORG_PATH)
photo_rot_path = os.path.join(path, PHOTO_ROT_PATH)

alphas = {}

for i in range(PHOTO_START, PHOTO_END):
    photo_a_path = os.path.join(photo_org_path, "atlantes_%d.jpg" % (i))
    photo_b_path = os.path.join(photo_org_path, "atlantes_%d.jpg" % (i+1))
    photo_a_file = open(photo_a_path, 'rb')
    photo_b_file = open(photo_b_path, 'rb')
    img_a = exif.Image(photo_a_file)
    img_b = exif.Image(photo_b_file)
    pnt_a = (dms2dec(img_a.get('gps_latitude'), img_a.get('gps_latitude_ref')), dms2dec(img_a.get('gps_longitude'), img_a.get('gps_longitude_ref')))
    pnt_b = (dms2dec(img_b.get('gps_latitude'), img_b.get('gps_latitude_ref')), dms2dec(img_b.get('gps_longitude'), img_b.get('gps_longitude_ref')))
    pnt_X = (pnt_a[0], pnt_b[1])
    dx = distance.great_circle(pnt_a, pnt_X).km * (1 if pnt_b[1] > pnt_a[1] else -1)
    dy = distance.great_circle(pnt_X, pnt_b).km * (1 if pnt_b[0] > pnt_a[0] else -1)
    alpha_rad = math.atan2(dy, dx)
    alpha_deg = math.degrees(alpha_rad)
    gamma = alpha_deg - beta + 180
    
    photo_cv = cv2.imread(photo_a_path, cv2.IMREAD_UNCHANGED)
    photo_rot_cv = rotate_image(photo_cv, gamma)
    photo_rot = os.path.join(photo_rot_path, "atlantes_%d.jpg" % (i))
    cv2.imwrite(photo_rot, photo_rot_cv)
    alphas[i] = alpha_deg

    photo_a_file.close()
    photo_b_file.close()
    
# Rotamos la última foto con el último ángulo calculado, ya que no tenemos la foto siguiente para
# hacer el cálculo con precisón.
photo_cv = cv2.imread(photo_b_path, cv2.IMREAD_UNCHANGED)
photo_rot_cv = rotate_image(photo_cv, gamma)
photo_rot = os.path.join(photo_rot_path, "atlantes_%d.jpg" % (PHOTO_END))
cv2.imwrite(photo_rot, photo_rot_cv)
alphas[PHOTO_END] = alpha_deg

# Guardamos los alphas calculados para utilizarlos más adelante.
json_object = json.dumps(alphas, indent = 4)
with open(os.path.join(path, "alphas_%d-%d.json" % (PHOTO_START, PHOTO_END)), "w") as text_file:
    text_file.write(json_object)

{
    "445": 20.513879965716082,
    "446": 19.41345026509236,
    "447": 18.294524243199554,
    "448": 17.15850726515467,
    "449": 16.003139921678297,
    "450": 14.834328663657553,
    "451": 13.647427236252012,
    "452": 12.447208908255812,
    "453": 11.23342849903942,
    "454": 10.00553882625628,
    "455": 8.768888413853645,
    "456": 7.520241494402383,
    "457": 6.261830192501049,
    "458": 4.9988872439592384,
    "459": 3.7258096384119677,
    "460": 2.4508813821963087,
    "461": 1.1723728219140284,
    "462": -0.10971242920438022,
    "463": -1.3913774180395584,
    "464": -2.6718409811483443,
    "465": -3.9506108419484276,
    "466": -5.223195410171575,
    "467": -6.494281618734384,
    "468": -7.755016923455526,
    "469": -9.007507789764139,
    "470": -10.252998211001144,
    "471": -11.483284934339991,
    "472": -12.706519557125631,
    "473": -13.913428205989913,
    "474": -15.105003835205226,
    "475": -16.282959708569336,
    "476": -17.443676396431307,
 